<a href="https://colab.research.google.com/github/yexf308/MAT592/blob/main/19_GMM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
from IPython.display import Image
import numpy.linalg as LA

Populating the interactive namespace from numpy and matplotlib


#Gaussian Mixture Models: Variations
##  Hidden Markov Models and Gaussian Mixture Models

- In hidden Markov models, we use "hidden state" instead of "clusters". Still we assume there are $K$ hidden states. 

- Some background of Markov chain is needed here. 
 -  Markovian property 
 - Property of the transition matrix $M_{ij}$ 
 - Invariant distribution of Markov chain. 



In [ ]:
display(Image(url='https://github.com/yexf308/MAT592/blob/main/image/HMM.png?raw=true', width=500))

Hidden Markov Models assumes that observed dataset $\{\mathbf{x}^{(i)}\}_{i=1}^N\subset \mathbb{R}^d$ are sampled through a generative process:

- Generate the sequence of states $\{z^{(t)}\}_{t=1}^N$ by a Markov chain whose transition matrix is $M\in \mathbb{R}^{K\times K}$ starting the initial distribution as the invariant distribution $\vec\pi$.

 **Assumption 1:** We assume the Markov chain has reached the stationary,  
 $$P(z^{(1)})=\vec\pi, \qquad \vec\pi = M\vec\pi$$

 **Assumption 2:** the state at time $t$, $z^{(t)}$, depends only on the state at the previous time $t-1$, $z^{(t-1)}$, i.e.,
 $$ P(z^{(t)}|z^{(1:t-1)})=P(z^{(t)}|z^{(t-1)})$$


- Given the state $z^{(t)}=c$, generate a point $\mathbf{x}^{(t)}$ from the associated multivariate Gaussian distribution $\mathcal{N}(\mu_c, \Sigma_c)$ with PDF. It is also called **emission distribution**:
$$p(\mathbf{x}|\mu_c, \Sigma_c)= \frac{1}{\sqrt{(2\pi)^d|\Sigma_c|}}\exp\left(-\frac{1}{2}(\mathbf{x}-\mu_c)^\top\Sigma_c^{-1}(\mathbf{x}-\mu_c)\right)$$
 **Assumption 3**: The observed data at time $t$, $\mathbf{x}^{(t)}$ depends only on the state at time $t$, $z^{(t)}$, i.e.,
 $$P(\mathbf{x}^{(t)}|z^{(1:t)}) =P(\mathbf{x}^{(t)}|z^{(t)})$$ 


Now the parameter $\theta=\{\pi_c, \mu_c, \Sigma_c,  M_{ij}\}_{c,i,j=1}^K$


## Likelihood of HMM

### Forward probability

The likelihood $\ell(\theta)=P(\mathbf{x}^{(1:N)}|\theta)$. Similarly,
$$P(\mathbf{x}^{(1:t)}|\theta)=\sum_{c=1}^K \underbrace{P(\mathbf{x}^{(1:t)}, z^{(t)}=c|\theta)}_{\triangleq\alpha_t(c)} $$

However, 
$$P(\mathbf{x}^{(1:N)}|\theta)\ne \Pi_{i=1}^N \sum_{c=1}^K \left(p(\mathbf{x}^{(i)}, z^{(i)}=c|\theta) \right)$$
Because these hidden state are not sampled independently!

$\alpha_t(c)$, is also called **forward probability**. 

\begin{align}
\alpha_t(c) &= P(\mathbf{x}^{(1:t)}, z^{(t)}=c|\theta) = \sum_{i=1}^K P(\mathbf{x}^{(1:t)}, z^{(t)}=c, z^{(t-1)}=i|\theta) \\
&=\sum_{i=1}^K P(\mathbf{x}^{(t)}| z^{(t)}=c, z^{(t-1)}=i, \mathbf{x}^{(1:t-1)}, \theta)\cdot P(z^{(t)}=c |z^{(t-1)}=i, \mathbf{x}^{(1:t-1)}, \theta)\cdot P(\mathbf{x}^{(1:t-1)}, z^{(t-1)}=i|\theta) \\
&= \sum_{i=1}^K P(\mathbf{x}^{(t)} |\mu_c, \Sigma_c) M_{ic}\alpha_{t-1}(i)
\end{align}

If we use linear algebra, the recurrsion formular has the following compact form, define $\vec\alpha_t = [\alpha_t(1), \dots, \alpha_t(K)]$, $B(\mathbf{x}^{(t)}) =\text{diag}([P(\mathbf{x}^{(t)} |\mu_1, \Sigma_1), \dots, P(\mathbf{x}^{(t)} |\mu_K, \Sigma_K)]) $, then
$$\vec\alpha_t =\vec\alpha_{t-1} M B(\mathbf{x}^{(t)}) $$
For convenience, set $\vec\alpha_0=\vec\pi$. So the forward probability and likelihood are
$$\vec\alpha_N= \vec \pi M B(\mathbf{x}^{(1)})M B(\mathbf{x}^{(2)})\dots M B(\mathbf{x}^{(N)})$$

$$ \ell(\theta)= \vec\alpha_N \mathbb{1}^\top$$
where $\mathbb{1}$ is a row of ones with length $K$. 




### Backward probability 
Define the **backward probability**, $\beta_t(c) = P(\mathbf{x}^{(t+1:N)}|z^{(t)}=c, \theta)$. Note 
\begin{align}
P(\mathbf{x}^{(t+1:N)}|z^{(t)}=c, \theta) &= \sum_{i=1}^K P(\mathbf{x}^{(t+1:N)}|z^{(t+1)}=i, \theta)P(z^{(t+1)}=i|z^{(t)}=c, \theta) \\
&=\sum_{i=1}^K P(\mathbf{x}^{(t+1)}|z^{(t+1)}=i, \theta)P(z^{(t+1)}=i|z^{(t)}=c, \theta)P(\mathbf{x}^{(t+2:N)}|z^{(t+1)}=i, \theta)
\end{align}
Then 
\begin{align}
\beta_t(c)  = \sum_{i=1}^K P(\mathbf{x}^{(t+1)}|\mu_i, \Sigma_i)M_{ci} \beta_{t+1}(i)
\end{align}
Define $\vec\beta_t =[\beta_t(1),\dots, \beta_t(K)]^T$.

For convenience, set $\vec\beta_T=[1, \dots, 1]^T$, the recursion formula has the following compact form.

$$\vec\beta_{t}=MB(\mathbf{x}^{(t+1)})\vec\beta_{t+1}$$

### Log-sum-exp trick
In practice, naively implementing forward algorithm will never work due to arithmetic underflow or overflow. 

Define $g_t(c)=\log \alpha_t(c)$, then 
\begin{align}
 g_t(c) = \log \alpha_t(c) = \log \sum_{i=1}^K \exp\left(l(z^{(t)}=c|z^{(t-1)}=i) + l(\mathbf{x}^{(t)}|z^{(t)}=c)+g_{t-1}(i)\right)
\end{align}
denoting $l=\log P$

To calculate this, one has to use **log-sum-exp** trick:

let’s suppose we would like to compute $\log\sum_{i=1}^m\exp(a_i)$, 

- choose $b=\max_{i}a_i$.

- \begin{align}
\log\sum_{i=1}^m\exp(a_i) =\log\sum_{i=1}^m\exp(a_i-b)\exp(b)  = b+\log\sum_{i=1}^m\exp(a_i-b)
\end{align}


## Expectation-Maximization(EM) algorithm: the general version

### The general version
The goal of EM is to find a maximum likelihood estimate (MLE) estimate in models involving hidden/latent/unobserved variables/data.

- Observed data $\mathbf{x}^{(1:N)}$.

- Model $(X, Z)\sim P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$. Here $z^{(1:N)}$ represents some collection of unobserved variables. Note EM works
best when $P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$ is an exponential family. 

- **Goal:** Find $\theta^*=\arg\max_{\theta} P(\mathbf{x}^{(1:N)}|\theta)$. Note the likelihood $P(\mathbf{x}^{(1:N)}|\theta) = \sum_{z^{(1:N)}}P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$. 

- **EM Algorithm:**
 - Initialize $\theta^1$. 

 - For $t=1, 2, \dots$ until some convergence criterion is met,

    - E-step: Compute the  auxiliary function

    \begin{align} \boxed{Q(\color{blue}\theta, \color{red}{\theta^t})=\mathbb{E}_{Z|X, \color{red}{\theta^t}}\left[\log P(X, Z|X=\mathbf{x}^{(1:N)} ,\color{blue}\theta)\right]   = \sum_{z^{(1:N)}} P(z^{(1:N)} |\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})\log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\color{blue} \theta)}
    \end{align}

    - M step: Solve for $\color{red}{\theta_{t+1}} = \arg\max_\theta Q(\color{blue}\theta, \color{red}{\theta^t})$

- In practice, we will be able to represent $Q(\color{blue}\theta, \color{red}{\theta^t})$  analytically as a function $\theta$. Moreover, we will
often be able to analytically maximize $Q(\color{blue}\theta, \color{red}{\theta^t})$. 

- It is usually a good idea to introduce some randomization into the initialization $\theta^1$. 





### Proof of correctness

Expectation-maximization works to improve $Q(\color{blue}\theta, \color{red}{\theta^t})$ rather than directly improving the log likelihood $\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)$. Here it is shown that improvements to the former imply improvements to the latter.

By Baye's rule, for any $z^{(1:N)}$ with non-zero probability  $P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta)$, 

$$\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) = \log P(\mathbf{x}^{(1:N)}, z^{(1:N)}|\color{blue}\theta) - \log P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta) $$

By multiplying both sides $P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})$ and summing over $z^{(1:N)}$, (which is equvilently, take the expectation over possible values of the hidden variable $Z$ under the current parameter estimate $\color{red}{\theta^t}$)
\begin{align}
\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)&=\sum_{z^{(1:N)}}P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})\log P(\mathbf{x}^{(1:N)}, z^{(1:N)}|\color{blue}\theta) -\sum_{z^{(1:N)}} P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})\log P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta) \\
& =Q(\color{blue}\theta, \color{red}{\theta^t}) +H(\color{blue}\theta|\color{red}{\theta^t})
\end{align}

when $\theta= \theta^t$, 
$$\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t})=Q(\color{red}{\theta^{t}}, \color{red}{\theta^t}) +H(\color{red}{\theta^t}|\color{red}{\theta^t}) $$

 Subtracting this last equation from the previous equation gives
\begin{align}
 \log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) -\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t}) &=  Q(\color{blue}\theta, \color{red}{\theta^t}) - Q(\color{red}{\theta^{t}}, \color{red}{\theta^t}) +H(\color{blue}\theta|\color{red}{\theta^t})-H(\color{red}{\theta^t}|\color{red}{\theta^t})   
  \end{align}
  \begin{align}
 \boxed{\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) -\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t}) \ge Q(\color{blue}\theta, \color{red}{\theta^t}) - Q(\color{red}{\theta^{t}}, \color{red}{\theta^t})}
 \end{align}

 It is due to the Gibbs' inequality that $H(\color{blue}\theta|\color{red}{\theta^t})\ge H(\color{red}{\theta^t}|\color{red}{\theta^t})$.

 So choose $\theta$ to maximize $Q(\color{blue}\theta, \color{red}{\theta^t})$ causes $\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)$ improves at least that much. 

## EM for HMM: Baum–Welch algorithm

### E-step
The joint probability $P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$ is 
\begin{align}
P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)&=P(z^{(1)})\cdot\Pi_{i=1}^{N-1}P(z^{(i+1)}|z^{(i)},\theta) \cdot\Pi_{j=1}^N P(\mathbf{x}^{(j)}|z^{(j)},\theta) \\
&=\pi_{z^{(1)}}\cdot\Pi_{i=1}^{N-1} M_{z^{(i)}z^{(i+1)}} \cdot \Pi_{j=1}^N P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}})
\end{align}
Then 
\begin{align}
\log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta) = \log \pi_{z^{(1)}} + \sum_{i=1}^{N-1} \log M_{z^{(i)}z^{(i+1)}} + \sum_{j=1}^N \log P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}})
\end{align}

The  auxiliary function $Q(\color{blue}\theta, \color{red}{\theta^t})$ is,

\begin{align}
Q(\color{blue}\theta, \color{red}{\theta^t}) &= \sum_{z^{(1:N)}} P(z^{(1:N)} |\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) \log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\color{blue}\theta)  \\
&= \sum_{z^{(1)}}\boxed{P(z^{(1)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})}\log \pi_{z^{(1)}}+\sum_{i=1}^{N-1}\sum_{z^{(i)}z^{(i+1)}} \boxed{P(z^{(i:i+1)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) }\log M_{z^{(i)}z^{(i+1)}} + \sum_{j=1}^N \sum_{z^{(j)}}\boxed{P(z^{(j)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})}\log P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}}) \\
& =  \sum_{k=1}^K\color{red}{\gamma_{k}^t(1)}\log \color{blue}\pi_{k}+\sum_{i=1}^{N-1}\sum_{k,c=1}^K \color{red}{\xi_{kc}^t(i)}\log\color{blue}{M_{kc}} + \sum_{j=1}^N \sum_{k=1}^K \color{red}{\gamma_{k}^t(j)}\log P(\mathbf{x}^{(j)}|\color{blue} {\mu_{k},\Sigma_{k}})
\end{align}

where $\color{red}{\gamma_{k}^t(i)} = P(z^{(i)}=k|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})$ and $\color{red}{\xi_{kc}^t(i)}= P(z^{(i)}=k, z^{(i+1)}=c|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) $. How to find $\gamma_{k}(i)$ and $\xi_{kc}(i)$?



### M-step 
 For the M-step, we need to find a value of $\theta$ maximizing $Q(\theta, \theta^t)$.  Fortunately, it turns out that we can often do this analytically. 

 - First, to maximize with respect to $\mu_k$ and $\Sigma_k$, i.e., 
 $$ 0= \nabla_{\mu_k}Q(\color{blue}\theta, \color{red}{\theta^t}) = \sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \nabla_{\mu_k} \log P(\mathbf{x}^{(j)}|\color{blue}{\mu_{k},\Sigma_{k}}) $$

  $$ 0= \nabla_{\Sigma_k}Q(\color{blue}\theta, \color{red}{\theta^t}) = \sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \nabla_{\Sigma_k} \log P(\mathbf{x}^{(j)}|\color{blue}{\mu_{k},\Sigma_{k}}) $$

  Then we have the analytical solution which is the solution of weighted MLE
  $$\mu_k^{t+1} = \frac{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \mathbf{x}^{(j)}}{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)}} $$

$$\Sigma_k^{t+1} =\frac{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \left[\mathbf{x}^{(i)}-\mu_k^{t+1}\right]\left[\mathbf{x}^{(i)}-\mu_k^{t+1}\right]^T }{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)}} $$

- Second, to maximize with respect to $\vec\pi$. The constraint is $\sum_{k}\pi_k = 1$. We can do this analytically using the method of Lagrange multipliers, the result is 
 
 $$ \pi_k^{t+1} =\frac{\gamma_k^t(1)}{\sum_{c=1}^K \gamma_c^t(1)} $$


- Third, to maximize with respect to the Markov transition matrix $M$. The contraint is $\sum_{c=1}^K M_{kc} =1$. We can do this analytically using
Lagrange multipliers as well. 

$$M_{kc}^{t+1}= \frac{\sum_{j=2}^N \xi_{kc}^t(j)}{\sum_{j=2}^N \sum_{c=1}^K \xi_{kc}^t(j)}. $$








### To find $\gamma_{k}(i)$ and $\xi_{kc}(i)$: Forward-Backward Algorithm 
\begin{align}
\gamma_{k}(i) & = P(z^{(i)}=k|\mathbf{x}^{(1:N)}, \theta)\\
&=\frac{P(z^{(i)}=k, \mathbf{x}^{(1:N)}|\theta)}{P(\mathbf{x}^{(1:N)}|\theta)}\\
&=\frac{P(z^{(i)}=k, \mathbf{x}^{(1:i)}|\theta)P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k, \theta)}{P(\mathbf{x}^{(1:N)}|\theta)} \\
&= \frac{\alpha_i(k)\beta_i(k)}{P(\mathbf{x}^{(1:N)}|\theta)}
\end{align}
This is because in the hidden markov model, 
$$ P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k, \mathbf{x}^{(1:i)})= P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k).$$

If we look at the MLE, $\gamma_k(i)$ shows up in both denorminator and numerator. So what matters is the relative ratio $\gamma_k(i)$ for different $i$. 

So we can renormalized $\vec\alpha_i$ and $\vec\beta_i$ at each $i$. 

Similarly, 
\begin{align}
\xi_{kc}(i)&= P(z^{(i)}=k, z^{(i+1)}=c|\mathbf{x}^{(1:N)} ,\theta) \\
&=\frac{P(z^{(i)}=k, z^{(i+1)}=c,\mathbf{x}^{(1:N)}|\theta )}{P(\mathbf{x}^{(1:N)}|\theta )} \\
& = \frac{P(z^{(i)}=k, \mathbf{x}^{(1:i)}|\theta)P(\mathbf{x}^{(i+2:N)}|z^{(i+1)}=c, \theta)P(z^{(i+1)}=c|z^{(i)}=k)P(\mathbf{x}^{(i+1)}|z^{(i+1)}=c)}{P(\mathbf{x}^{(1:N)}|\theta )} \\
&=\frac{\alpha_i(k)\beta_{i+1}(c)M_{kc}P(\mathbf{x}^{(i+1)}|\mu_c, \Sigma_c)}{P(\mathbf{x}^{(1:N)}|\theta)}
\end{align}



